In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/finance_course/2021/lesson3')

## Overnight Index Swap

* Interest rate swaps (IRS) are usually used to mitigate the risks of fluctuations of varying interest rates, or to benefit from lower rates. 

* Overnight Index Swaps (OIS) are a particular kind of IRS which pay a floating coupon, determined by overnight rate fixings over the reference periods, against a fixed coupon.  
  * We will always look at these products from the point of view of the **receiver of the floating leg**.

* An OIS is defined by:
  * a notional amount $N$;
  * a starting date $d_0$;
  * a sequence of payment dates $d_1,...,d_n$;
  * a fixed rate $K$.

* For simplicity in the following we assume the fixed and floating legs to have the same notional and payment dates, although this is not necessarily always the case in practice.


### OIS Valuation

* To evaluate the NPV of such products the cash flows of each leg have to be calculated; then sum their discounted values.


#### Floating leg

* The floating leg pays a cash flow determined as follows:

$$f_{\mathrm{float},~i} = N \Bigg\{\prod_{d=d_{i-1}}^{d=d_i-1}\Big(1+r_{\mathrm{O/N}}(d)\cdot\frac{1}{360}\Big) -1 \Bigg\}$$

* This formula is valid for OIS swaps in EUR since the $\frac{1}{360}$ fraction appears because EONIA rates are quoted using the ACT/360 day-count convention.
  * Other currencies might have different conventions.  
  * In addition we are making the simplifying assumption of ignoring weekends and holidays:
    * each overnight rate is valid for only one day.

* The sum of the discounted expected values of these cash flows is

$$\mathrm{NPV}_{\mathrm{float}} = \sum_{i=1}^{n}D(d_i)\mathbb{E}[f_{\mathrm{float},~i}]$$

* where $D(d)$ is the discount factor with expiry $d$. 

* On the other hand, by definition 

$$\mathbb{E}[f_{\mathrm{float},~i}] = N\cdot\Big(\frac{D_{\mathrm{OIS}}(d_{i-1})}{D_{\mathrm{OIS}}(d_{i})} - 1\Big)$$

$$\mathrm{NPV}_{\mathrm{float}} = N\cdot \sum_{i=1}^{n}D(d_i) \Big(\frac{D_{\mathrm{OIS}}(d_{i-1})}{D_{\mathrm{OIS}}(d_{i})} - 1\Big)$$

* where $D_{\mathrm{OIS}}(d)$ is the discount factor implied by OIS prices (we will see how to derive it).

* Since the correct curve to discount OIS is the overnight index itself we have that $D = D_{\mathrm{OIS}}$ so the NPV simplifies to

$$\begin{equation}
  \begin{split}
    \mathrm{NPV}_{\mathrm{float}} & = N\cdot\sum_{i=1}^{n}[D(d_{i-1}) - D(d_i)] =  \\
    &= N\cdot[(D(d_{0}) - D(d_{1})) + (D(d_{1}) - D(d_{2})) + ... + (D(d_{n-1}) - D(d_{n}))]\\
    &= N \cdot [D(d_0) - D(d_n)]
  \end{split}
\end{equation}
$$

#### Fixed leg
* The calculation for the fixed leg is simpler: each cash flow is  equal to

$$f_{\mathrm{fixed},~i}=N\cdot K\cdot \frac{d_i - d_{i-1}}{360}$$

* So the NPV of the fixed leg is

$$\mathrm{NPV}_{\mathrm{fixed}} = N\cdot K\cdot \sum_{i=1}^{n}D_{\mathrm{OIS}}(d_{i})\frac{d_i - d_{i-1}}{360}$$


### Discount Factor Determination from Market Quotes

* Our ultimate goal is to take a series of Overnight Index Swap quotations, and determine the discount factors implied by their prices. 
  * To do this we will build a class to represent OIS and compute its value given particular discount curve; 
  * then we will use this class, with a numerical optimizer, to *invert* the relation which connects the NPV to the discount curve so that the implied discount factors can be determined from OIS prices (market quotes).

In [ ]:
# define OIS class
                    

#### Example

In [ ]:
# test the class with a fake curve
from datetime import date
from finmarkets import DiscountCurve

ois = OvernightIndexSwap(
            # the notional, one million
            1e6,
            # the list of payment dates
            [date(2020, 10, 21), date(2021, 1, 21),
             date(2021, 4, 21), date(2021, 7, 21),
             date(2021, 10, 21)],
            # the fixed rate, 2.5%
            0.025)

# fake discount curve
curve = DiscountCurve([date(2020, 10, 21), date(2021, 6, 1),
                       date(2022, 1, 1)],
                      [1.0, 0.98, 0.82])

print ("OIS NPV: {:.2f}".format(ois.npv(curve)))

## Bootstrapping Technique 

* We would like to determine a discount curve starting from the market quotes of a set of Overnight Index Swaps with different maturities.

* The employed technique is called bootstrapping.
  * This is the ABC of financial mathematics, since you always need a
discount curve to price a contract. 
  * We concentrate on EONIA swaps in order to build an EUR discount curve.
* The asumption underlying bootstrapping is that market quotes represent the **fair** prices of the OIS so they make the swap NPVs null.
  * The fair price is an estimate of what a willing buyer would pay a willing seller for a given asset, assuming both have a reasonable knowledge of the asset's worthness.


### Building OIS Instances

* The first step involves getting data, the swap market quotes. 
  * This is not easy since EONIA swap market is over the counter (OTC) and it’s not straightforward to access it. 
  * Anyway in [ois_data.xlsx](https://github.com/matteosan1/finance_course/raw/develop/libro/input_files/ois_data.xlsx) it is available 
a dataset of swap quotes (which actually are rates rather prices) for our needs.


In [ ]:
# read and inspect ois_data.xlsx
import pandas as pd


* Let’s build a 15 months swap instance using above input data. 
  * Be careful when doing this operation and double check the units of rates, quotes, etc...
  * For example quotes are expressed in percent so you need to multiply by 0.01 before using them;
  * another detail to check is that 15 months quote is not the fifteenth entry in the $\tt{DataFrame}$ (actually it is the twelfth).

In [ ]:
# create an OvernightIndexSwap with a market quote
from finmarkets import generate_dates


### Bootstrap Algorithm

* In finance, bootstrap is a method for constructing a (zero-coupon) yield curve from the prices of a set of coupon-bearing products (e.g. bonds and swaps). 
* The term structure of spot rates is obtained from the bond yields by solving for them recursively (by *forward substitution*): 
  * this iterative process is what is called the bootstrap method. 
* The usefulness of bootstrap is that using only few selected products, it is possible to derive forward and spot rates for all maturities.

* Let’s consider the following example which can be solved, at least partially, analytically.
* Select few bonds (coupon of 4\%, 5\%, 6\%, 7\% and 8\% respectively) with
maturities ranging from 1 to 5 years, each having a value of €100 and traded at par (traded at its face value). 

* To determine the zero-coupon yield curve proceed as follows:
  1. at the end of first year the discounted cash flow of the first bond is €104 (principal plus the coupon) times the discount factor, so the implied 1 year rate 

$$100 = \cfrac{104}{(1 + S_{1y})}\implies S_{1y} =  104/100 - 1 = 4\%$$

  2. at the end of second year the sum of the cash flows of the second bond can be compared to its trading price to compute the 2-year spot rate $S_{2y}$ (using the previously derived value of $S_{1y}$)

$$100 = \cfrac{5}{(1 + S_{1y})} + \cfrac{105}{(1 + S_{2y})^{2}}$$
<br>
$$
100 = 5 / (1 + 0.04) + 105 / (1 + S_{2y})^{2}\qquad\Rightarrow\qquad S_{2y}^2  + 2 S_{2y}  - 0.103030 = 0 $$

* This second order equation can be solved either by hand or with `numpy.roots`.






In [1]:
import numpy as np
np.roots([1, 2, -0.103030])

array([-2.05025235,  0.05025235])

$$S_{2y} = - 1 \pm \sqrt{1 + 0.103030} = \begin{cases}-2.05023 \\ 0.0502\end{cases}$$

* From the third year on we should obtain equation of third order or more:
  * they are not easily analitically solvable. 
  
* For example the equation for the fifth bond at the fifth year:

$$100 = \cfrac{8} {(1 + S_{1y})} + \cfrac{8} {(1 + S_{2y})^{2}}+ \cfrac{8} {(1 + S_{3y})^{3}} + \cfrac{8} {(1 + S_{4y})^{4}} + \cfrac{108} {(1 + S_{5y})^{5}}$$

* Assuming we have already determined the previous rates:

| years | coupon rate | bond price | rate|
|:----:|:----:|:----:|:----:|
|1 | 1.00 % | 100 | 4.00% |
|2 | 2.00 % | 100 | 5.02% |
|3 | 3.00 % | 100 | 6.08% |
|4 | 4.00 % | 100 | 7.19% |
|5 | 5.00 % | 100 | ??? |

* Previous equation can be solved numerically using for example Brent's algorithm.

![](https://drive.google.com/uc?id=1AmCejDf3ifMz9Zc6XOHeAk-5Tc23Zbrw)

In [ ]:
# find zeros of previous eq. with brentq
from scipy.optimize import brentq


* The very same mechanism can be generalized to more maturities to get a more detailed yield curve: 

$$\begin{equation*}
\begin{cases}
f_1(S_1, p_1) = 0 \\
f_2(S_1, S_2, p_2) = 0 \\
f_3(S_1, S_2, S_3, p_3) = 0 \\
f_4(S_1, S_2, S_3, S_4, p_4) = 0 \\
\cdots
\end{cases}
\end{equation*}
$$
* where $S_i$ are the unknown spot rates and $p_i$ the market quotes of the considered products. 

* The iterative procedure we have applied before exploits the first equation to find $S_1 = f_1^{-1}(p_1)$, the second to find $S_2 = f_2^{-1}(S_1, p_2)$ and so on...

* This algorithm works since each equation will determine exactly one free spot rate which is not already determined by the others.


### Bootstrap as Minimization Problem

* Instead of iteratively finding the solution of each equation as before, we could:
  * define a vector of spot rates $\mathbf{S} = (S_1, S_2, S_3,\ldots)$;
  * seek for a particular $\mathbf{\hat{S}}$ which solves the following equation:

$$F = f_1^2(\hat{S}_1,p_1) + f_2^2(\hat{S}_1, \hat{S}_2,p_2) + f_3^2(\hat{S}_1, \hat{S}_2, \hat{S}_3,p_3) + f_4^2(\hat{S}_1, \hat{S}_2, \hat{S}_3, \hat{S}_4,p_4) + \ldots = 0$$

* Under this terms the bootstrap technique can be considered as a minimization problem;
  * indeed we need to find $\mathbf{\hat{S}}$ which makes $F$ zero, or at least *minimize* it making $F$ as close as possible to 0.
  * Notice that each $f_i$ is squared since we want all of them to be minimized at the same time and not only $F$ globally (without the squared there may be cancellation effects between the terms of the sum).


### Minimization Algorithm

* A minimization algorithm follows these steps:
  1. define an *objective function* i.e. the function that is actually minimized to reach our goal;
  2. set the initial value of the unknown parameters and their range of variability;
  3. the minimizer will compute the objective function value;
  4. then it will move the parameter values in such a way to find a smaller value of the objective function (e.g. following the derivative w.r.t. each parameter);
  5. if there are contraints the step above will take them into account;
  6. steps 4 and 5 are repeated until further variations of the $\mathbf{x}$ values won’t change significantly the objective function (i.e. we have found a minimum of the function so the minimisation process is completed !).

* In $\tt{python}$ the algorithm is implemented in $\tt{scipy.optimize.minimize}$

### Example 
* Find the dimensions that will minimize the costs to manufacture a cylindrical can of volume $330~\mathrm{cm}^3 (33~\mathrm{cl})$.

![](https://drive.google.com/uc?id=1LQaX8j10nq1KgRu4RdR4Ade96fsMNxa6)

* Minimize the costs means the company needs to reduce the can surface, given the required volume.

$$ S(r, h) = 2\pi rh + 2\cdot(\pi r^2) $$

* The volume is fixed to $330~\mathrm{cm}^3$ so we can remove $h$ from the previous equation:

$$ V = \pi r^2 h = 330\quad\implies h = \cfrac{330}{\pi r^2} $$

* The surface function to be minimized is:

$$ S(r) = 2\pi rh + 2\cdot(\pi r^2) = \cfrac{2\cdot 330}{r} + 2\cdot(\pi r^2)$$



In [ ]:
from math import pi


* Set the limits to our unknown variable and its initial value:

* Run the minimization:

In [2]:
from scipy.optimize import minimize


* To minimize the cost the company should produce cans with a radius of about 3.745 cm.

### Example with Constraint

* You need to fence a rectangular field. 
  * If we look at the field from above the cost of the vertical sides are €10/m, the cost of the bottom is €2/m and the cost of the top is €7/m. 
  * If we have €700 determine the dimensions of the field that will maximize the enclosed area.

![](https://drive.google.com/uc?id=1JhL79-u0uDs8nkcEtletOWypdKWWZDBc)

* In this example there are two differences w.r.t before:
  * we want to maximize a quantity (not minimize);
  * there is a contraint (we have a limited amount of money).

* The objective is to **maximize** the enclosed area $A$ so we can **minimize** the quantity $-A$. 
* Further define length and width of the field with $\tt{x[0]}$ and $\tt{x[1]}$ (items of the list $\tt{x}$).

* Set the boundaries for length and width and their initial values (1 m each):

* We need to impose the constraint on the money.
* This is done by defining a function that computes the money spent with the fence and compare it to €700.
* The constraint is passed to the minimizer as a dictionary which has two keys: 
  * $\tt{type}$ with value $\tt{eq}$ (like equality) since we want to spend all of our available money so the fence has to cost €700

$$\mathrm{fence~cost} = l\cdot10 + l\cdot10 + w\cdot2 + w\cdot7 = 700$$
$$l\cdot10 + l\cdot10 + w\cdot2 + w\cdot7 - 700 = 0$$

  * $\tt{fun}$ whose value is the constraint function. 

* Run the minimizer.

* So the field will come out $17.5$ m long and $38.9$ m wide.

## Local Minima

In [ ]:
import numpy as np
from math import pi

def func(x):
    return np.cos(3*pi*x)/x

![](https://drive.google.com/uc?id=1NarHa2FYJxDT6nq_sm6H57PUJcwHBxxx)

In [ ]:
from scipy.optimize import minimize
x0 = [1.1]
bounds = [(0.01, 2)]

r = minimize(func, x0, bounds=bounds)
print (r)

In [ ]:
from scipy.optimize import minimize
x0 = [0.5]
bounds = [(0.01, 2)]

r = minimize(func, x0, bounds=bounds)
print (r)

### Back to OIS Example

* The general idea is to find the discount curve $\mathcal{C}$ such that it prices as much correctly as possible each OIS by minimizing the sum of their NPVs (our $f_i$):

$$\mathrm{min}_{\mathcal{C}} \Big\{\sum_{i=1}^{n}\mathrm{NPV}^2(\mathrm{OIS}_i, \mathcal{C})\Big\}$$

* A discount curve is characterized by pillar dates and the corresponding discount factors.
  * So far it hasn't been specified any constraint on the number of pillar dates of the discount curve $\mathcal{C}$ we are going to find (the number of unknown variables of the optimization problem). 
  * **In practice, therefore, it makes sense to choose the pillar dates in such a way that there are exactly the right number of degrees of freedom in the optimization to match data, so equal number of pillars and market quotes.** 
  * Hence the natural choice is to choose the pillar dates of the discount curve equal to the set of expiry dates of the swaps.

* The optimization problem becomes

$$ F= \mathrm{min}_{\mathbf{x}} \Big\{\sum_{i=1}^{N}\mathrm{NPV}^2(\mathrm{OIS}_i, \mathcal{C}(\mathbf{d}, \mathbf{x}))\Big\}\qquad (f_i^2 = \mathrm{NPV}^2(\mathrm{OIS}_i, \mathcal{C}(\mathbf{d}, \mathbf{x})))$$

* with $\mathbf{d}$ vector of pillar dates equal to the expiry dates of the OIS swaps, and $\mathbf{x}$ vector of unknown discount factors.

* First create the swaps according to all the available market quotes and also the pillar dates of our final discount curve:

In [ ]:
# creates the OIS from market quotes
from finmarkets import generate_dates

observation_date = date.today()
pillar_dates = [observation_date]
swaps = [] # container of the OIS objects 


* Define the objective function: the sum of the squared NPVs of the OIS 

In [ ]:
# define objective function
import numpy as np
from finmarkets import DiscountCurve


* Set the initial value of the discount factors ($x_i$) to 1 with a range of variability $[ 0.01, 10]$, in addition the first element of the list, today’s discount factor, will be fixed to 1 (variability $[1, 1]$)

In [ ]:
# set boundaries and guess values


* Launch the minimizer to find the discount factors ($\mathbf{x}$)

In [ ]:
# minimize


* Some diagnostic number/plot.

In [ ]:
# print initial and final objective function values


![](https://drive.google.com/uc?id=17VFztegQzIOh1IMKf49BJlVXO51F_n5P)
![](https://drive.google.com/uc?id=1poBs7n51TC_rrXww1mzbqBkdHH5ZoyoD)

* Finally we can create the discount curve implied by the market quote of our swaps. 

$$ \mathrm{df} = e^{-rt} \quad\implies \mathrm{log(df)} = -rt\quad\implies r = -\mathrm{log(df)}/t$$

In [ ]:
# create the discount curve with our factors
from math import log
from dateutil.relativedelta import relativedelta
from finmarkets import DiscountCurve


In [ ]:
from matplotlib import pyplot as plt
